### 
NAMAE  : RAHINI DEVI S

ROLLNO : 225229129

Lab - 12 (Image corpus creation and Transfer Learning in CNN)

## Steps
### 1. Import the necessary libraries

In [1]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

### 2. Initialize some parameters

In [2]:
now = datetime.datetime.now #get current time

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size =3

### 3. Partition MNIST dataset

In [3]:
# data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create 2 datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

### 4. Define the 'feature' layers

In [4]:
feature_layers =[ 
    Conv2D(filters, kernel_size = (3,3), activation = 'relu', padding='valid', input_shape = (img_rows,img_cols,1)),
    Conv2D(filters, kernel_size = (3,3), activation = 'relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten() 
]

### 5. Define the 'classification' layers

In [5]:
classification_layers = [
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(num_classes, activation = 'softmax')
]

### 6. Define a Sequential model

In [6]:
model = Sequential(feature_layers + classification_layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                        

### 7. Create a function train_model()

In [7]:
import numpy as np
import pandas as pd

In [8]:
def train_model(model,train,test,num_classes):
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)
    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')
    train[0] /= 255
    test[0] /= 255
    print(train[0].shape)
    print(test[0].shape)
    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values
    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values
    model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])
    train_start_time=now()
    model.fit(train[0],train[1],epochs=epochs,verbose=2)
    print('\n')
    print('Training time: ', (now() - train_start_time))
    score=model.evaluate(test[0],test[1],verbose=0)
    print('test loss ',score[0])
    print('test accuracy ',score[1])

### 8. Training on digits 5 to 9

In [9]:
train_model(model,
            [x_train_gte5, y_train_gte5],
            [x_test_gte5, y_test_gte5], num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 68s - loss: 1.5716 - accuracy: 0.3048 - 68s/epoch - 74ms/step
Epoch 2/5
919/919 - 70s - loss: 1.4846 - accuracy: 0.4810 - 70s/epoch - 76ms/step
Epoch 3/5
919/919 - 69s - loss: 1.3744 - accuracy: 0.6030 - 69s/epoch - 75ms/step
Epoch 4/5
919/919 - 72s - loss: 1.2413 - accuracy: 0.6716 - 72s/epoch - 79ms/step
Epoch 5/5
919/919 - 68s - loss: 1.0938 - accuracy: 0.7182 - 68s/epoch - 74ms/step


Training time:  0:05:47.449069
test loss  0.9451504945755005
test accuracy  0.8656654953956604


### 9. Freeze Feature Layers

In [10]:
for l in feature_layers:
  l.trainable = False

### 10. Print Summary

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                        

### 11. Training for digits 0 to 4 based on digits 5 to 9

In [12]:
train_model(model,
            [x_train_lt5, y_train_lt5],
            [x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 37s - loss: 1.4501 - accuracy: 0.4228 - 37s/epoch - 38ms/step
Epoch 2/5
957/957 - 35s - loss: 1.2316 - accuracy: 0.5691 - 35s/epoch - 37ms/step
Epoch 3/5
957/957 - 36s - loss: 1.0550 - accuracy: 0.6980 - 36s/epoch - 38ms/step
Epoch 4/5
957/957 - 35s - loss: 0.9052 - accuracy: 0.7776 - 35s/epoch - 36ms/step
Epoch 5/5
957/957 - 34s - loss: 0.7931 - accuracy: 0.8157 - 34s/epoch - 36ms/step


Training time:  0:02:57.382437
test loss  0.6528003215789795
test accuracy  0.8929752707481384


### 12. Reversing the training process

In [13]:
Reverse_model = Sequential(feature_layers + classification_layers)
Reverse_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                      

In [14]:
train_model(Reverse_model,
            [x_train_gte5, y_train_gte5],
            [x_test_gte5, y_test_gte5], num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 36s - loss: 1.0890 - accuracy: 0.6080 - 36s/epoch - 39ms/step
Epoch 2/5
919/919 - 35s - loss: 0.9664 - accuracy: 0.6837 - 35s/epoch - 38ms/step
Epoch 3/5
919/919 - 32s - loss: 0.8772 - accuracy: 0.7346 - 32s/epoch - 35ms/step
Epoch 4/5
919/919 - 32s - loss: 0.8104 - accuracy: 0.7637 - 32s/epoch - 35ms/step
Epoch 5/5
919/919 - 32s - loss: 0.7533 - accuracy: 0.7878 - 32s/epoch - 35ms/step


Training time:  0:02:47.854202
test loss  0.6407154202461243
test accuracy  0.8494136929512024


In [15]:
train_model(Reverse_model,
            [x_train_lt5, y_train_lt5],
            [x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 37s - loss: 0.8805 - accuracy: 0.7221 - 37s/epoch - 38ms/step
Epoch 2/5
957/957 - 31s - loss: 0.7515 - accuracy: 0.7800 - 31s/epoch - 32ms/step
Epoch 3/5
957/957 - 32s - loss: 0.6662 - accuracy: 0.8213 - 32s/epoch - 34ms/step
Epoch 4/5
957/957 - 31s - loss: 0.5980 - accuracy: 0.8430 - 31s/epoch - 33ms/step
Epoch 5/5
957/957 - 34s - loss: 0.5468 - accuracy: 0.8606 - 34s/epoch - 36ms/step


Training time:  0:02:45.380834
test loss  0.4377923011779785
test accuracy  0.9028993844985962
